In [4]:
import sys
import os

script_path = os.path.dirname(".")
project_root = os.path.abspath(os.path.join(script_path, "../.."))
print(f"Project_root: {project_root}")
sys.path.append(project_root)

import pandas as pd
from analysis.utils.constants import stats_2020_path, stats_2021_path
stats_2021 = pd.read_csv(stats_2021_path).dropna()
stats_2020 = pd.read_csv(stats_2020_path).dropna()
data_set = {2020:stats_2020, 2021:stats_2021}

import panel as pn
pn.extension()

from analysis.player_rating import (
    get_players_pool, get_player_ratings, combine_player_data, get_expected_salary)
from analysis.utils.constants import stats_counts
players_pool = get_players_pool(n_teams=14, n_players=13, over_write=400)
ratings_2021 = get_player_ratings(stats_2021, players_pool=players_pool)
ratings_2020 = get_player_ratings(stats_2020, players_pool=players_pool)
player_ratings = combine_player_data(data_2020=ratings_2020, data_2021=ratings_2021)
player_stats = combine_player_data(
    data_2020=stats_2020.set_index("name"), data_2021=stats_2021.set_index("name")
)

for n_expensive in [5, 7, 9, 11, 13]:
    player_ratings[f'salary_{n_expensive}'] = get_expected_salary(player_ratings,    
                                                      players_per_team=13,
                                                      n_teams= 14,
                                                      one_dollar_rank= 13*n_expensive).round(2)


Project_root: /Users/kliu/my_projects/espn-api


In [5]:
#* create dashboard components
preview_cols = ["name", "PTS", "BLK", "STL", "AST", "REB", "3PTM", 
                "FGR", "FTR", "overall", "salary_5", "salary_13"]
preview_ratings = pn.widgets.DataFrame(player_ratings.reset_index()[preview_cols].round(2), 
                                       widths={'name':150}, height=450)
player_search_box = pn.widgets.AutocompleteInput(
    name='Player Search', options=list(player_ratings.index),
    value='Anthony Davis',
    placeholder='Start with First name (capitalized)')

candidates_columns = stats_counts + ["FGR", "FTR"]
x_axis_col = pn.widgets.Select(name='x_axis', value='FGR',
                               options=candidates_columns)
y_axis_col = pn.widgets.Select(name='y_axis', value='FTR',
                               options=candidates_columns)

from analysis.visualization import plot_player_ratings_scatter
color_col = "overall"
size_col = "overall"
plot_opts = dict(width=500, height=400, title="Stats from 2021", tools=['tap', 'hover'])
@pn.depends(x_axis_col.param.value, y_axis_col.param.value)
def pn_scatter(x_col, y_col):
    scatter = plot_player_ratings_scatter(player_ratings.reset_index()
                                          , x_col=x_col, y_col=y_col, **plot_opts)
    return scatter

@pn.depends(player_search_box.param.value)
def pn_player_stats(player_name):
    ratings = player_ratings.reset_index()
    player_row = ratings["name"] == player_name
    if player_row.sum() > 0:
        this_rating = ratings[player_row]
    else:
        this_rating = pd.DataFrame({}, columns=ratings.columns)
    rating_frame = pn.widgets.DataFrame(this_rating.round(2).T, width=150, widths={"index":20})
    stats_cols = ['position', 'pro_team', 'PTS', 'BLK', 'STL', 'AST', 'REB', 'TO', 'FGM',
                  'FGA', 'FTM', 'FTA', '3PTM', '3PTA', 'FG%', 'FT%', '3PT%', 'MPG', 'MIN',
                  'stats_type']
    this_stats = player_stats[stats_cols].round(2).loc[player_name].to_frame()
    stats_frame = pn.widgets.DataFrame(this_stats.dropna(), width=150, widths={"index":30})
    return pn.Row(rating_frame, stats_frame)


from analysis.player_rating import get_similar_players, get_expected_salary
import hvplot.pandas

def plot_player_heatmap(player_ratings:pd.DataFrame, **heatmap_opts):
    heatmap = player_ratings[
        ["PTS", "BLK", "STL", "AST", "REB", "3PTM", "FGR", "FTR"]
    ].hvplot.heatmap(**heatmap_opts)
    return heatmap

@pn.depends(player_search_box.param.value)
def pn_similar_players(player_name):
    if player_name is None:
        results = player_ratings.head(10)
    else:
        results = get_similar_players(player_name, player_ratings=player_ratings, score_method="euclidean")
    return plot_player_heatmap(results, 
                               rot=45,width=500, height=400, 
                               cmap="jet", clim=(-2, 3))

dashboard = pn.Column(
    pn.Row(pn.Column(preview_ratings, x_axis_col, y_axis_col), 
           pn.Column(player_search_box, pn_player_stats)),
    pn.Row(pn_scatter, pn_similar_players)
)


In [6]:
dashboard.servable();


Column
    [0] Row
        [0] Column
            [0] DataFrame(height=450, value=              ..., widths={'name': 150})
            [1] Select(name='x_axis', options=['PTS', 'BLK', ...], value='FGR')
            [2] Select(name='y_axis', options=['PTS', 'BLK', ...], value='FTR')
        [1] Column
            [0] AutocompleteInput(name='Player Search', options=['Anthony Davis', ...], placeholder='Start with First n..., value='Anthony Davis')
            [1] ParamFunction(function)
    [1] Row
        [0] ParamFunction(function)
        [1] ParamFunction(function)

In [8]:
# player_ratings.to_csv("draft_ratings_14_teams.csv")

# draft_kit = player_ratings.reset_index()[['name', 'overall', "salary_5", "salary_13"]].round(2).copy()
# draft_kit['drafted_by'] = 'free_agent'
# draft_kit['price'] = 0
# draft_kit.to_csv("draft_kit.csv")